3. 레이블링\
객체 단위 분석: 객체를 분할하여 특징 분석. 객체 위치, 크기 정보
roi 추출, 모양 분석 등\
레이블링: 서로 연결되어 있는 객체 픽셀에 고유 번호 할당. 영역 기반 모양 분석. 레이블맵, 바운딩 박스, 픽셀 개수, 무게중심 좌표 반환\
외곽선 검출: 객체 외곽선 좌표 검출. 외곽선 기반 모양분석. 다양한 외곽선 처리 함수에서 활용(근사화, 컨벡스 헐 등)\

1) 레이블링\
동일 객체에 속한 모든 픽셀에 고유 번호 할당\
보통 이진 영상에서 수행\

픽셀 연결 관계\
4-이웃 연결 관계\
8-이웃 연결 관계\

레이블링 함수\
cv2.connectedComponents(src, connectivity, ltype)=> retval, labels\
src : 8비트 1채널 영상\
connectivity: 4 또는 8 . 기본값 8\
ltype: labels 타입. 기본값은 cv2.CV_32S\
retval: 객체 개수(레이블 개수 ) N반환. 카운팅은 0부터 이고 0은 배경 객체, 실제 객체 개수는 N-1개\

cv2.connectedComponentsWithStats(src,connectivity,ltype)=>\
retval, labels, stats, centroids\
src: 8비트 1채널 영상\
connectivity: 4또는 8. 기본값은 8\
ltype: labels타입. 기본값은 cv2.CV_32S\
stats: 추출된 객체의 위치, 가로세로 길이, 면적 등의 정보\
centroids:무게 중심 정보\

In [4]:
import sys 
import numpy as np
import cv2 

src = cv2.imread('keyboard.bmp',cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()
    
_, src_bin = cv2.threshold(src,0,255, cv2.THRESH_OTSU)

cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

for i in range(1, cnt):
    (x,y,w,h,area) = stats[i]
    
    if area <20:
        continue
        
    cv2.rectangle(dst,(x,y,w,h),(0,255,255))
    



cv2.imshow('src',src)
cv2.imshow('src_bin',src_bin)
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

2) 외곽선 검출\
객체 외곽선 좌표 추출\
외부 외곽선 & 내부 외곽선 => 외곽선 계층 구조 표현 가능\

cv2.findContours(img,mode,method,offset) => contours, hierarchy\

mode: contours 찾는 방법 지정\
cv2.RETR_EXTERNAL : 가장 바깥 경계선만 찾음\
cv2.RETR_LIST: 모든 컨투어 탐색.hierarchy 관계 구성 않음\
cv2.RETR_CCOMP: 모든 컨투어 탐색, hierarchy 관계 2단계 까지 구성\
cv2.RETR_TREE: 모든 컨투어 탐색. 모든 hierarchy 관계 구성\

method: 컨투어 좌표와 관련된 설정\
cv2.CHAIN_APPROX_NONE: 모든 컨투어 좌표 저장\
cv2.CHAIN_APPROX_SIMPLE:컨투어 라인을 그릴수 있는 좌표만 저장\
cv2.CHAIN_APPROX_TC89_L1: 컨투어 좌표 찾는 알고리즘\
cv2.CHAIN_APPROX_TC89_KCOS:컨투어 좌표 찾는 알고리즘\

offset:좌표 이동 옵셋. 기본값(0,0)\

contours: 검출된 외곽선 좌표. len(contours)는 전체 길이이므로 \
외곽선 좌표의 개수가 됨.\

contours[i].shape = (k,1,2) k는 한 객체 외곽선 좌표 개수, \
contours[i].dtype= numpy.int32\

hierarchy: 외곽선 계층정보. \
numpy.ndarray.shape=(1,N,4),\
type = numpy.int32\
hierarchy[0,i,0]~ hierarchy[0,i,3]이 순서대로\
next, prev, child, parent. 외곽선 인덱스를 가리킴\
해당 외곽선이 없으면 -1\

